In [1]:
import os
import sys

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [ ]:
from core.backtesting.optimizer import BacktestingConfig, BaseStrategyConfigGenerator
from hummingbot.strategy_v2.executors.position_executor.data_types import TrailingStop
from controllers.directional_trading.macd_mt_dca import MacdMTDCAControllerConfig
from decimal import Decimal
import optuna

class MACDMTConfigGenerator(BaseStrategyConfigGenerator):
    """
    Strategy configuration generator for MACD MT optimization.
    """
    async def generate_config(self, trial) -> BacktestingConfig:
            
        # paramemeters when dynamic = False
        stop_loss = trial.suggest_float("stop_loss", 0.01, 0.1, step=0.01)
        trailing_stop_activation_price = trial.suggest_float("trailing_stop_activation_price", 0.004, 0.02, step=0.001)
        trailing_delta_ratio = trial.suggest_float("trailing_delta_ratio", 0.05, 0.1, step=0.01)
        trailing_stop_trailing_delta = trailing_stop_activation_price * trailing_delta_ratio
        dca_spread_1 = trial.suggest_float("dca_spread_1", 0.01, 0.04, step=0.01)       
            
        dynamic_order_spread = False
        dynamic_target =  False


        bb_interval = trial.suggest_categorical("bb_interval", ["1m", "5m", "15m", "1h"])
        bb_length = trial.suggest_int("bb_length", 50, 200, step=50)
        bb_std = trial.suggest_float("bb_std", 1.5, 3, step=0.5)
        min_stop_loss=Decimal("0.007")
        max_stop_loss=Decimal("0.1")
        min_trailing_stop=Decimal("0.0025")
        max_trailing_stop=Decimal("0.03")
        #min_distance_between_orders=Decimal("0.01"),
        
        #stop_loss = trial.suggest_float("stop_loss", 0.1, 1, step=0.1)
        #dca_spread_1 = trial.suggest_float("dca_spread_1", 0.25, 1, step=0.25)       
        #trailing_stop_activation_price = trial.suggest_float("trailing_stop_activation_price", 0.25, 1, step=0.25)
        #trailing_delta_ratio = trial.suggest_float("trailing_delta_ratio", 0.05, 0.1, step=0.01)
        #trailing_stop_trailing_delta = trailing_stop_activation_price * trailing_delta_ratio
        trailing_stop = TrailingStop(
            activation_price=Decimal(trailing_stop_activation_price),
            trailing_delta=Decimal(trailing_stop_trailing_delta)
        )

        
        dca_amount_1 = trial.suggest_float("dca_amount_1", 1.0, 3.0, step=1.0)  
        max_executors_per_side = trial.suggest_int("max_executors_per_side", 1, 3, step=1)
        macd_interval_1 = trial.suggest_categorical("macd_interval_1", ["1m", "5m", "15m", "1h"])
        macd_signal_type_1 = trial.suggest_categorical("macd_signal_type_1", ["mean_reversion_1", "mean_reversion_2", "trend_following"])
        macd_fast_1 = trial.suggest_int("macd_fast_1", 9, 59, step=10)
        macd_slow_1 = trial.suggest_int("macd_slow_1", 21, 201, step=10)
        macd_signal_1 = trial.suggest_int("macd_signal_1", 10, 100, step=10)
        macd_number_of_candles_1 = trial.suggest_int("macd_number_of_candles_1", 1, 6, step=2)
        macd_interval_2 = trial.suggest_categorical("macd_interval_2", ["1m", "5m", "15m", "1h"])
        macd_signal_type_2 = trial.suggest_categorical("macd_signal_type_2", ["mean_reversion_1", "mean_reversion_2", "trend_following"])
        macd_fast_2 = trial.suggest_int("macd_fast_2", 9, 59, step=10)
        macd_slow_2 = trial.suggest_int("macd_slow_2", 21, 201, step=10)
        macd_signal_2 = trial.suggest_int("macd_signal_2", 10, 100, step=10)
        macd_number_of_candles_2 = trial.suggest_int("macd_number_of_candles_2", 1, 6, step=2)
        cooldown_time =  trial.suggest_int("cooldown_time", 60*60, 60*60*5, step=60*60)
        take_profit = 100
        time_limit = 60 * 60 * 24 * 2
        total_amount_quote = 100
        executor_refresh_time = 60*2
        
        # Apply logical constraints, pruning if violated
        if (macd_fast_1 >= macd_slow_1 or macd_fast_2 >= macd_slow_2):
            raise optuna.TrialPruned()



        # Create the strategy configuration
        config = MacdMTDCAControllerConfig(
            connector_name="binance_perpetual",
            trading_pair="WLD-USDT",
            macd_interval_1 = macd_interval_1,
            macd_signal_type_1 = macd_signal_type_1,
            bb_interval = bb_interval,
            bb_length = bb_length,
            bb_std = bb_std,
            dynamic_order_spread = dynamic_order_spread,
            dynamic_target = dynamic_target,
            min_stop_loss = min_stop_loss,
            max_stop_loss = max_stop_loss,
            min_trailing_stop = min_trailing_stop,
            max_trailing_stop = max_trailing_stop,
            #min_distance_between_orders = min_distance_between_orders,
            macd_fast_1 = macd_fast_1,
            macd_slow_1 = macd_slow_1,
            macd_signal_1 = macd_signal_1,
            macd_number_of_candles_1 = macd_number_of_candles_1,
            macd_interval_2 = macd_interval_2,
            macd_signal_type_2 = macd_signal_type_2,
            macd_fast_2 = macd_fast_2,
            macd_slow_2 = macd_slow_2,
            macd_signal_2 = macd_signal_2,
            macd_number_of_candles_2 = macd_number_of_candles_2,
            dca_amounts = [Decimal("1"), dca_amount_1],
            dca_spreads = [Decimal("-0.00000001"), dca_spread_1],
            total_amount_quote=Decimal(total_amount_quote),
            take_profit=Decimal(take_profit),
            stop_loss=Decimal(stop_loss),
            trailing_stop=trailing_stop,
            time_limit=time_limit,
            max_executors_per_side=max_executors_per_side,
            executor_refresh_time = executor_refresh_time,
            cooldown_time=cooldown_time,
        )

        # Return the configuration encapsulated in BacktestingConfig
        return BacktestingConfig(config=config, start=self.start, end=self.end)

In [ ]:
from core.backtesting.optimizer import StrategyOptimizer
import datetime

start_date = datetime.datetime(2024, 10, 15)
end_date = datetime.datetime(2024, 10, 30)
config_generator = MACDMTConfigGenerator(start_date=start_date, end_date=end_date)

optimizer = StrategyOptimizer(root_path=root_path)
await optimizer.optimize(
    study_name="macd_mt_fixed_pct_2_quincena_oct",
    config_generator=config_generator,
    n_trials=5000,
)

[I 2024-11-01 23:15:50,923] Using an existing study with name 'macd_mt_v5_test_with_optimizer' instead of creating a new one.
/home/sapel/miniconda3/envs/quants-lab/lib/python3.10/site-packages/optuna/distributions.py:704: UserWarning: The distribution is specified by [1, 6] and step=2, but the range is not divisible by `step`. It will be replaced by [1, 5].
  warnings.warn(
2024-11-01 23:15:52,240 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7efeae6078b0>
2024-11-01 23:15:52,246 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x7efeae81f2e0>, 44026.322606028)]']
connector: <aiohttp.connector.TCPConnector object at 0x7efeae604a30>
2024-11-01 23:15:58,555 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7efefc1a8b80>
2024-11-01 23:15:58,561 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSe

An error occurred during optimization: 


Traceback (most recent call last):
  File "/home/sapel/workspace/quants-lab/core/backtesting/optimizer.py", line 311, in _async_objective
    backtesting_config = await config_generator.generate_config(trial)
  File "/tmp/ipykernel_1115489/3249385910.py", line 66, in generate_config
    raise optuna.TrialPruned()
optuna.exceptions.TrialPruned
2024-11-01 23:17:56,166 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7efecc3600d0>
2024-11-01 23:17:56,168 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x7efeae81ece0>, 44150.249816917)]']
connector: <aiohttp.connector.TCPConnector object at 0x7efecc3603d0>
2024-11-01 23:17:57,998 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7efecc360460>
2024-11-01 23:17:58,000 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x7efeae81ece

An error occurred during optimization: 


2024-11-01 23:20:05,135 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7efeae61de70>
2024-11-01 23:20:05,140 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x7efeae76dd80>, 44279.216418185)]']
connector: <aiohttp.connector.TCPConnector object at 0x7efeae61cd30>
2024-11-01 23:20:08,880 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7efecc3c0520>
2024-11-01 23:20:08,884 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x7efecc222c80>, 44282.958074984)]']
connector: <aiohttp.connector.TCPConnector object at 0x7efecc3c04f0>
2024-11-01 23:20:20,671 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7efeae61c5e0>
2024-11-01 23:20:20,672 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHa

...

2024-11-01 23:23:17,324 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7efecc360850>
2024-11-01 23:23:17,328 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x7efecc350940>, 44471.401498103)]']
connector: <aiohttp.connector.TCPConnector object at 0x7efecc360ac0>
2024-11-01 23:23:20,780 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7efecc7b3910>
2024-11-01 23:23:20,782 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x7efece6e0160>, 44474.867020578)]']
connector: <aiohttp.connector.TCPConnector object at 0x7efecc7b37c0>
2024-11-01 23:23:30,928 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7efecc3603a0>
2024-11-01 23:23:30,929 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHa

An error occurred during optimization: 


2024-11-01 23:24:56,603 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7efecc323850>
2024-11-01 23:24:56,608 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x7efece6e0400>, 44570.6771754)]']
connector: <aiohttp.connector.TCPConnector object at 0x7efecc321a80>
2024-11-01 23:25:06,266 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7efecc4c30d0>
2024-11-01 23:25:06,270 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x7efecc5e2c20>, 44580.345641248)]']
connector: <aiohttp.connector.TCPConnector object at 0x7efecc4c37c0>
2024-11-01 23:25:08,560 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7efecc59e3e0>
2024-11-01 23:25:08,565 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHand

An error occurred during optimization: 


2024-11-01 23:25:42,973 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7efecc4e3250>
2024-11-01 23:25:42,975 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x7efece6e0160>, 44617.056171348)]']
connector: <aiohttp.connector.TCPConnector object at 0x7efecc4e33a0>
2024-11-01 23:25:45,665 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7efece6d4f70>
2024-11-01 23:25:45,670 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x7efeae8a9240>, 44619.746406875)]']
connector: <aiohttp.connector.TCPConnector object at 0x7efece6d5d50>
2024-11-01 23:25:54,557 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7efece6d4a60>
2024-11-01 23:25:54,559 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHa

In [3]:

from core.backtesting.optimizer import StrategyOptimizer

optimizer = StrategyOptimizer(root_path=root_path)
optimizer.launch_optuna_dashboard()